In [77]:
from bs4 import BeautifulSoup
import requests
import itertools as it
from tidylib import tidy_document
import tidylib
import re

tidylib.BASE_OPTIONS = {
    "indent": 1,           # Pretty; not too much of a performance hit
    "tidy-mark": 0,        # No tidy meta tag in output
#     "wrap": 0,             # No wrapping
    "alt-text": "",        # Help ensure validation
    "doctype": 'strict',   # Little sense in transitional for tool-generated markup...
    "force-output": 1,     # May not get what you expect but you will get something
    'drop_empty_paras': False
    }

url = "http://www.lispworks.com/documentation/HyperSpec/Body/06_a.htm"
html_doc = requests.get(url).text

html_doc2 = html_doc.replace("<P>", "<p></p>")
html_doc2
soup = BeautifulSoup(html_doc2, 'html.parser')

In [55]:
url = "http://www.lispworks.com/documentation/HyperSpec/Body/06_a.htm"
base_url = "http://www.lispworks.com/documentation/HyperSpec/Body/"

def parse_clhs(url):
    html_doc = requests.get(url).text
    # Can't be used because some documents have regular, clsoed <p> tags
#     html_doc = html_doc.replace("<P>", "<p></p>") 
    # This is for closing unclosed <p> tags, that mess up beautiful soup
    tidy_doc = tidy_document(html_doc, options={'drop_empty_paras': False, 'force_output': True})[0]
    soup = BeautifulSoup(tidy_doc, 'html.parser')
    return soup

def remove_non_content(soup):
    head = [tag for tag in soup.body.hr.find_previous_siblings()]
    for tag in head:
        tag.decompose()
    tail = [tag for tag in soup.body.hr.find_next_siblings()]
    for tag in tail:
        tag.decompose()
    return soup

def get_content(soup):
    without_head = [tag for tag in soup.body.hr.find_next_siblings()]
    content = list(it.takewhile(lambda x: str(x) != '<hr/>', without_head))
#     content = without_head
    return content

def get_base_url(url):
    return re.findall(".*\/", url)[0]

def extract_link(a_tag, base_url):
    return base_url + a_tag.a['href']

def extract_full_link(a_tag):
    return extract_link(a_tag, base_url)

In [6]:
clhs = parse_clhs(url)
filtered = [tag for tag in get_content(clhs) if tag.a]
filtered
links = list(map((lambda x: base_url + x.a['href']), filtered))
links
# # first_html = requests.get(links[0]).text
# first_html = parseClhs(links[0])
# first_html = getContent(first_html)
# first_html
# links
# for tag in getContent(clhs):
#     print(tag)
#     print("\n\n")
# bool(getContent(clhs)[2].a)

['http://www.lispworks.com/documentation/HyperSpec/Body/06_aa.htm',
 'http://www.lispworks.com/documentation/HyperSpec/Body/06_ab.htm',
 'http://www.lispworks.com/documentation/HyperSpec/Body/06_ac.htm',
 'http://www.lispworks.com/documentation/HyperSpec/Body/06_ad.htm',
 'http://www.lispworks.com/documentation/HyperSpec/Body/06_ae.htm',
 'http://www.lispworks.com/documentation/HyperSpec/Body/06_af.htm',
 'http://www.lispworks.com/documentation/HyperSpec/Body/06_ag.htm',
 'http://www.lispworks.com/documentation/HyperSpec/Body/06_ah.htm',
 'http://www.lispworks.com/documentation/HyperSpec/Body/06_ai.htm']

In [58]:
def replace_content_recursive(url):
    clhs = parse_clhs(url)
    content = remove_non_content(clhs)
    h2s = [tag for tag in content if tag.name == 'h2' and tag.a]
    base_url = get_base_url(url)
    for h2 in h2s:
        print("working on: " + h2.a['href'])
        content_tag_list = replace_content_recursive(base_url + h2.a['href']).reverse()
        for tag in content_tag_list:
            h2.insert_after(tag)
    return content

def replace_headline_with_content(url):
    print("Inside replace_headline_with_content for: " + url)
    clhs = parse_clhs(url)
    content = get_content(clhs)
    h2s = [tag for tag in content if tag.name == 'h2' and tag.a]
    base_url = get_base_url(url)
#     return list(map(replace_headline_with_content, [get_base_url(url) + tag.a['href'] for tag in h2s]))
#     return [get_base_url(url) + tag.a['href'] for tag in h2s]
#     return list(map(replace_content_recursive, h2s))
#     return list(map(lambda x: (x, replace_headline_with_content(x)) ,[get_base_url(url) + tag.a['href'] for tag in h2s]))
    for h2 in h2s:
        print("working on: " + h2.a['href'])
# #         h2.insert_after(replace_content_recursive(base_url + h2.a['href']))
#         content_tag_list = replace_content_recursive(base_url + h2.a['href']).reverse()
#         for tag in content_tag_list:
#             h2.insert_after(tag)
        
    return clhs

print("printing the soup.")
soup = replace_headline_with_content("http://www.lispworks.com/documentation/HyperSpec/Body/06_a.htm")
with open("clhs.html", "w") as f:
    print("started writing.")
    f.write(str(soup))

print("Finish!")

printing the soup.
Inside replace_headline_with_content for: http://www.lispworks.com/documentation/HyperSpec/Body/06_a.htm
working on: 06_aa.htm


TypeError: 'NoneType' object is not callable

In [60]:
clhs = parse_clhs("http://www.lispworks.com/documentation/HyperSpec/Body/06_aaa.htm")
content = get_content(clhs)
h2s = [tag for tag in content if tag.name == 'h2' and tag.a]
[get_base_url(url) + tag.a['href'] for tag in h2s]
replace_headline_with_content("http://www.lispworks.com/documentation/HyperSpec/Body/06_aaa.htm")

Inside replace_headline_with_content for: http://www.lispworks.com/documentation/HyperSpec/Body/06_aaa.htm
working on: 06_aaaa.htm


TypeError: 'NoneType' object is not callable

In [155]:
clhs = parse_clhs(url)
content = get_content(clhs)
h2s = [tag for tag in content if tag.name == 'h2' and tag.a]

def replace_tags_with_content(tags, base_url):
    for tag in tags:
        link =  extract_link(tag, base_url)
        print(link)
        content = get_content(parse_clhs(link))
        yield content
#         print(content)
#         break
        
# list(map(replace_content_recursive, h2s))
# list(replace_tags_with_content(h2s, base_url))

data = ''
with open('new.html', 'r') as file:
    data = file.read().replace('\n', '')


# parse_clhs("http://www.lispworks.com/documentation/HyperSpec/Body/06_ac.htm")
# requests.get("http://www.lispworks.com/documentation/HyperSpec/Body/06_ac.htm").text
html_doc = requests.get("http://www.lispworks.com/documentation/HyperSpec/Body/06_ac.htm").text
# html_doc = requests.get("http://www.lispworks.com/documentation/HyperSpec/Body/06_ab.htm").text
# html_doc = html_doc.replace("<P>", "<p></p>") 
# tidy_doc = html_doc
# Can't be used because some documents have regular, clsoed <p> tags
#     html_doc = html_doc.replace("<P>", "<p></p>") 
# This is for closing unclosed <p> tags, that mess up beautiful soup
# tidy_doc = tidy_document(html_doc, options={'drop_empty_paras': False, 'force_output': True})[0]
# if not tidy_doc:
#     tidy_doc = html_doc
# soup = BeautifulSoup(tidy_doc, 'html.parser')
# soup = BeautifulSoup(tidy_doc, 'html.parser')
# get_content(soup)
# soup
# tidy_doc

# tidylib.BASE_OPTIONS = {
#     "indent": 1,           # Pretty; not too much of a performance hit
#     "tidy-mark": 0,        # No tidy meta tag in output
# #     "wrap": 0,             # No wrapping
#     "alt-text": "",        # Help ensure validation
#     "doctype": 'strict',   # Little sense in transitional for tool-generated markup...
#     "force-output": 1,     # May not get what you expect but you will get something
#     'drop_empty_paras': False
#     }
# tidy_doc = tidy_document(html_doc, options={'drop_empty_paras': False, 'force_output': True})
# tidy_doc


# tidy_document(html_doc, options={'drop_empty_paras': False, 'force_output': True})
# html_doc
# get_content(BeautifulSoup(tidy_doc[0], 'html.parser'))

filtered = [tag for tag in get_content(BeautifulSoup(tidy_doc[0], 'html.parser')) if tag.name == 'h2' and tag.a]
filtered

[<h2>
 <a href="06_aca.htm" rel="CHILD">6.1.3.1 Examples of COLLECT
       clause</a>
 </h2>, <h2>
 <a href="06_acb.htm" rel="CHILD">6.1.3.2 Examples of APPEND
       and NCONC clauses</a>
 </h2>, <h2>
 <a href="06_acc.htm" rel="CHILD">6.1.3.3 Examples of COUNT
       clause</a>
 </h2>, <h2>
 <a href="06_acd.htm" rel="CHILD">6.1.3.4 Examples of MAXIMIZE
       and MINIMIZE clauses</a>
 </h2>, <h2>
 <a href="06_ace.htm" rel="CHILD">6.1.3.5 Examples of SUM
       clause</a>
 </h2>]

In [58]:
html_doc = requests.get(url).text
tidy_doc = tidy_document(html_doc, options={'drop_empty_paras': False})
soup = BeautifulSoup(tidy_doc[0], 'html.parser')
# getContent(parseClhs(url))
# tidy_doc[0]
get_base_url(url)

'http://www.lispworks.com/documentation/HyperSpec/Body/'

In [110]:
clhs = parse_clhs("http://www.lispworks.com/documentation/HyperSpec/Body/06_a.htm")
tidy_doc = tidy_document(clhs)
body = clhs.body

for tag in body.hr.find_previous_siblings():
    tag.decompose()
body.hr.decompose()
for tag in body.hr.find_next_siblings():
    tag.decompose()
body.hr.decompose()
body.find_all()[2:]


[<h2>
 <a href="06_aa.htm" rel="CHILD">6.1.1 Overview of the Loop
       Facility</a>
 </h2>, <a href="06_aa.htm" rel="CHILD">6.1.1 Overview of the Loop
       Facility</a>, <p></p>, <h2>
 <a href="06_ab.htm" rel="CHILD">6.1.2 Variable Initialization
       and Stepping Clauses</a>
 </h2>, <a href="06_ab.htm" rel="CHILD">6.1.2 Variable Initialization
       and Stepping Clauses</a>, <p></p>, <h2>
 <a href="06_ac.htm" rel="CHILD">6.1.3 Value Accumulation
       Clauses</a>
 </h2>, <a href="06_ac.htm" rel="CHILD">6.1.3 Value Accumulation
       Clauses</a>, <p></p>, <h2>
 <a href="06_ad.htm" rel="CHILD">6.1.4 Termination Test
       Clauses</a>
 </h2>, <a href="06_ad.htm" rel="CHILD">6.1.4 Termination Test
       Clauses</a>, <p></p>, <h2>
 <a href="06_ae.htm" rel="CHILD">6.1.5 Unconditional Execution
       Clauses</a>
 </h2>, <a href="06_ae.htm" rel="CHILD">6.1.5 Unconditional Execution
       Clauses</a>, <p></p>, <h2>
 <a href="06_af.htm" rel="CHILD">6.1.6 Conditional Execution
     